# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [29]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# Imports and connect to a Cassandra instance 

from cassandra.cluster import Cluster
try:
    cluster = Cluster()

# Connect Cassandra cluster and assign to session object variable
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [33]:
# Create  Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
# Connect to Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [35]:

#Create table song_session table partitioned on sessionid and itemInsession
""" The following statement creates Song_session1 table partitioned on session_id and ItemInsession
      Song_Session1 table modeled to store session_id , ItemInsession,artist,title and song length partitioned by session_id and ItemInsession values """
query ="CREATE TABLE IF NOT EXISTS song_session1"
query=query + "(session_id int, itemInSession int ,artist text,song_title text,song_length double, PRIMARY KEY(session_id,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [36]:
#set csv filename to file variable
file = 'event_datafile_new.csv'
#loops through the all records in the event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #  Assign the INSERT statements into the `query1` variable
        query1 = "INSERT INTO song_session1 (session_id,itemInSession,artist,song_title,song_length)"
        query1 = query1 + " VALUES (%s,%s,%s,%s,%s)"
        #  Assign csv column   positon value for each column in the INSERT statement.

        session.execute(query1, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
#  Below SELECT statement to retive the data based session_id and IteminSession values
query = "Select artist,song_title,song_length from song_session1 where session_id = 338 and itemInSession = 4 "
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    #print artist,song and songlength values from song_session table
    print (row.artist, row.song_title, row.song_length)
    """Above query retuns following result set
       Faithless Music Matters (Mark Knight Dub) 495.3073

    """

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [38]:
"""This following statement creates table usersession with userid,sessionid,iteminsession,artist,sing and user fields. This table has two primary key columns
     userid and sessionid and one cluser field iteminsession"""
# Create table user_session to store userid,sessionid,itemInsession,artist,song,user full name partitioned by userid and session id and ordered bt iteminsession
query2 ="CREATE TABLE IF NOT EXISTS user_session"
query2=query2 + "(userId int,session_id int , itemInSession int ,artist text,song text, user text,PRIMARY KEY((userId,session_id),itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)


                    

In [39]:
# Set filename of csv to file variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #  Belo statement assign the INSERT statements into the `query2` variable
        query2 = "INSERT INTO user_session (userId,session_id,itemInSession,artist,song,user)"
        query2 = query2 + " VALUES (%s,%s,%s,%s,%s,%s)"
        #concat user first name and last name and assign to userName
        userName=line[1] + ' '+ line[4]
       
        session.execute(query2, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9],userName))


                      
                    

In [40]:
"""Below select query to retrive all the artist, song and user data from user session table where userId = 10 and session_id = 182 sorted by IteminSession"""
  #  SELECT statement to retrive artist,song,user data based on userid and sessionid
query2 = "Select itemInSession,artist,song,user from user_session where userId = 10 and session_id = 182 "
try:
    rows=session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    #print artist,song and user fullname
    print (row.artist, row.song, row.user)
        

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [41]:
""" Below statement creates table user_Songs with song and user fields and primary key defined on both song and user fields to return unique  values"""

#Create table user_songs with sing and user fields partitioned by song(Primary Key)

query3 ="CREATE TABLE IF NOT EXISTS user_songs"
query3=query3 + "(song text, user text,PRIMARY KEY(song,user))"
try:
    session.execute(query3)
except Exception as e:
    print(e)


### Drop the tables before closing out the sessions

In [42]:
#  set up the CSV file to file variable
file = 'event_datafile_new.csv'
#Loop condition to to read daa from CSV file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

#  assign the INSERT statements  to query3 variable
        query3 = "INSERT INTO user_songs (song,user)"
        query3 = query3 + " VALUES (%s,%s)"
        #concat user first name and last name and assign to userName
        userName=line[1] + ' '+ line[4]
        session.execute(query3, (line[9],userName))

In [43]:
#   SELECT statement to retive users  data who listened to the song 'All Hands Against His Own'
query3 = "Select song,user from user_songs where song = 'All Hands Against His Own'"
try:
    rows=session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    print (row.song,row.user)


All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Sara Johnson
All Hands Against His Own Tegan Levine


### Close the session and cluster connection¶

In [44]:
# Drop statement to drop  song_session1,user_session,tables
query1 = "DROP  TABLE IF EXISTS song_session1"
query2 = "DROP  TABLE IF EXISTS user_session"
query3 = "DROP  TABLE IF EXISTS user_songs"


try:
    rows = session.execute(query1)
    rows = session.execute(query2)
    rows = session.execute(query3)

except Exception as e:
    print(e)


In [45]:
#close the session and cluster
session.shutdown()
cluster.shutdown()